In [1]:
import json
import time
import hmac
import hashlib
import requests
from time import sleep

In [2]:
API_KEY = "Your_API_Key_Here"
SECRET_KEY = "Your_Secret_Key_Here"

In [ ]:
while True:
    try:
        data = requests.get('https://api.crypto.com/v2/public/get-ticker?instrument_name=ETH_USDT')
        response = data.json()
        eth_first = response['result']['data']['k']
    except exception as e:
        print('Error obtaining first ETH data')
    
    sleep(300)
    
    try:
        data = requests.get('https://api.crypto.com/v2/public/get-ticker?instrument_name=ETH_USDT')
        response = data.json()
        eth_second = response['result']['data']['k']
    except exception as e:
        print('Error obtaining second ETH data')
    
    percent = (((float(eth_second) - float(eth_first)) * 100) / float(eth_first))
    
    if percent != 5:
        print('The trade requirement was not satisfied. The percentage move is at ',percent)
        sleep(0.1)
        continue
        
    elif percent == 5:
        try:
            req = {
             "id": 11,
             "method": "private/create-order",
             "params": {
               "instrument_name": "BTC_USDT",
               "side": "BUY",
               "type": "MARKET",
               "quantity": 5,
               "client_oid": "my_order_0002",
               "time_in_force": "GOOD_TILL_CANCEL",
             },
             "nonce": int(time.time() * 1000)
            }

            paramString = ""

            if "params" in req:
                  for key in sorted(req['params']):
                    paramString += key
                    paramString += str(req['params'][key])

            sigPayload = req['method'] + str(req['id']) + req['api_key'] + paramString + str(req['nonce'])

            req['sig'] = hmac.new(
              bytes(str(secret_key), 'utf-8'),
              msg=bytes(sigPayload, 'utf-8'),
              digestmod=hashlib.sha256
            ).hexdigest()
            
            order = requests.post('https://uat-api.3ona.co/v2/private/create-order', json=req, headers={'Content-Type':'application/json'})
        except exception as e:
            print('Error placing order at {}'.format(pd.Timestamp.now()))
            
        print(order)
        sleep(2)

        order_id = order['result']['id']

        req = {
            "id": 11,
            "method": "private/get-order-detail",
            "params": {
                "order_id": order_id
            },
             "nonce": int(time.time() * 1000)
            }

        try:
            order_check = requests.get('https://uat-api.3ona.co/v2/private/create-order', json=req, headers={'Content-Type':'application/json'})
        except exception as e:
            print('Error checking order')

        if order_check['result']['order_info']['status'] == 'FILLED' or order_check['result']['order_info']['status'] == 'ACTIVE':
            print ('Order placed at {}'.format(pd.Timestamp.now()))
            break

        else:
            print('Order was canceled at {}'.format(pd.Timestamp.now()))
            break